In [243]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Import Data

In [244]:
df_covid = pd.read_csv("covid19_news.csv")

In [245]:
df_covid.sample(10)

,doc_content,doc_date,doc_id,doc_source,doc_title,doc_url
1067,closeVideoDemocrats use coronavirus fears to t...,02/27/2020,1026378,Fox,"Ingraham spars with ex-Clinton aide, hits Dems...",https://www.foxnews.com/media/laura-ingraham-s...
3567,"TOPEKA, Kan. (AP) — A second COVID-19 death in...",03/22/2020,1038354,US News,Kansas Sees Second Death From COVID-19 as Case...,https://www.usnews.com/news/best-states/kansas...
2299,\nPresident Donald Trump declared a national s...,03/13/2020,1035375,Breitbart,Donald Trump Declares National Emergency over ...,http://feedproxy.google.com/~r/breitbart/~3/x2...
758,DUBAI (Reuters) - Two Iranians have died in ho...,02/19/2020,1020147,US News,Two Iranians Die After Testing Positive for Co...,https://www.usnews.com/news/world/articles/202...
3556,By Asif Shahzad\nISLAMABAD (Reuters) - A Pakis...,03/22/2020,1038470,US News,Pakistan Doctor Dies Amid Strike Threat Over L...,https://www.usnews.com/news/world/articles/202...
2920,"By MARY ESCH, Associated Press\nALBANY, N.Y. (...",03/16/2020,1036320,US News,Schools and Bars Shut as Virus Deaths Tick up ...,https://www.usnews.com/news/best-states/new-yo...
621,closeVideoSan Diego adult becomes the 13th con...,02/12/2020,1014717,Fox,Nearly 200 Georgia residents being monitored f...,https://www.foxnews.com/us/nearly-200-georgia-...
2978,"By JOHN RABY, Associated Press\nCHARLESTON, W....",03/17/2020,1037063,US News,All 50 States Now Have Virus Cases: West Virgi...,https://www.usnews.com/news/best-states/west-v...
1631,"By RYAN TARINELLI, Associated Press\nALBANY (A...",03/06/2020,1031572,US News,Suburban Rabbi Among New York's 44 Cases of CO...,https://www.usnews.com/news/best-states/new-yo...
1780,WASHINGTON (Reuters) - The U.S. Army said on S...,03/08/2020,1032755,US News,"U.S. Army Restricts Travel to Italy, South Kor...",https://www.usnews.com/news/us/articles/2020-0...


In [246]:
df_covid.doc_content[3180]

' (CNN)Former President Barack Obama thanked the health care workers fighting the novel coronavirus pandemic, spotlighting the story of one family impacted by combating the virus and urging Americans to show similar commitment to the common good.\n"We owe a profound debt of gratitude to all our health professionals and everybody who\'ll be on the front lines of this pandemic for a long while," Obama tweeted on Tuesday, sharing the story of a family in which the father, a doctor, had opted to self-quarantine away from his family while treating coronavirus patients.\n"They\'re giving everything," Obama added. "May we all model our own behavior on their selflessness and sacrifice as we help each other through this."\nObama referenced the circumstances of Rachel Patzer, a medical professor who tweeted on Tuesday that she and her husband, an emergency department physician, "made the difficult decision for him to isolate & move into our garage apartment for the foreseeable future as he conti

In [247]:
df_covid.doc_content[2509]

'closeVideoCoronavirus drive-thru testing site opens in New Rochelle, NYDrivers will remain in their cars and be swabbed by workers in hazmat suits; Alex Hogan reports from the scene.New Yorkers woke up Friday in the city that never sleeps to a fast-spreading coronavirus pandemic — and to an empty Times Square, no Broadway shows, no Knicks and Nets basketball, no concerts or other large gatherings and deserted\xa0office buildings.Subways and commuter trains were running but with significantly fewer riders.Aqueduct racetrack near JFK Airport told railbirds to stay home but was still planning to run a full card of horse races Friday afternoon.Thursday\'s dizzying series of coronavirus-related shutdowns included some\xa0of New York City’s cultural jewels: the Metropolitan Opera, Metropolitan Museum of Art, the American Museum of Natural History and Carnegie Hall.\n      A pedestrian wearing a face mask stops in Times Square, Thursday, March 12, 2020, in New York. New York City Mayor Bill 

In [248]:
df_covid.doc_id[2509]

1035069

## doc content preprocessing

In [249]:
dt_fox = df_covid[df_covid.doc_source == 'Fox']

In [250]:
len(dt_fox)

330

In [251]:
dt_fox.head()

,doc_content,doc_date,doc_id,doc_source,doc_title,doc_url
14,BEIJING -- China reported Monday a sharp rise ...,01/20/2020,993253,Fox,China counts sharp rise in coronavirus cases d...,https://www.foxnews.com/health/china-counts-sh...
43,closeVideoSen. Tom Cotton links coronavirus sp...,01/22/2020,995686,Fox,Tom Cotton: China needs to answer for 'malfeas...,https://www.foxnews.com/media/tom-cotton-coron...
47,closeVideoTrump took swipe at Democratic candi...,01/22/2020,995366,Fox,Trump says US has plan to contain coronavirus,https://www.foxnews.com/health/trump-says-us-h...
54,closeVideoChina expands quarantine measures am...,01/23/2020,996817,Fox,Who are the coronavirus victims? What to know ...,https://www.foxnews.com/health/who-are-the-cor...
56,closeVideoWorld Health Organization officials ...,01/23/2020,996824,Fox,WHO says China's coronavirus 'too early' for e...,https://www.foxnews.com/health/who-china-coron...


In [268]:
index = list(df_covid.doc_content.index.values)
index[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [269]:
#dt_fox.doc_content[index[329]]

In [296]:
K = []

for i, d in enumerate(index):
    #print(i,d)
    text = df_covid.doc_content[d]
    para_lst = text.split('\n\s*')
    
    para_lst = pd.DataFrame(para_lst)
    para_lst.columns = ['paragraph_str']
    er = df_covid.doc_id[i]
    para_lst = pd.concat([para_lst], keys=[er], names=['doc_id'])
    K.append(pd.DataFrame())
    
    K[i] = para_lst.paragraph_str.apply(lambda x: pd.Series(x.split('.'))).stack()
    K[i] = K[i].to_frame().rename(columns={0:'sentence_str'})
    
    K[i].index.names = ['doc_id','paragraph_num','sentence_num']
    K[i] = K[i].sentence_str.apply(lambda x: pd.Series(x.split())).stack()
    
    K[i] = K[i].to_frame().rename(columns={0:'token_str'})
    K[i].index.names = ['doc_id','paragraph_num','sentence_num','token_num']
    
    K[i]['term_str'] = K[i].token_str.replace('\W+', '', regex=True).str.lower()

#print (K[13], K[7])

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [298]:
K[2509]

token_str  \
doc_id  paragraph_num sentence_num token_num                          
1035069 0             0            0          closeVideoCoronavirus   
                                   1                     drive-thru   
                                   2                        testing   
                                   3                           site   
                                   4                          opens   
...                                                             ...   
                      43           2                          Press   
                                   3                    contributed   
                                   4                             to   
                                   5                           this   
                                   6                         report   

                                                           term_str  
doc_id  paragraph_num sentence_num token_num                         
1035069 0             0            0          closevideocoronavirus  
                                   1                      drivethru  
                                   2                        testing  
                                   3                           site  
                                   4                          opens  
...                                                             ...  
                      43           2                          press  
                                   3                    contributed  
                                   4                             to  
                                   5                           this  
                                   6                         report  

[843 rows x 2 columns]

In [272]:

df_total = pd.concat(K,axis = 0)

In [273]:
df_total

token_str  term_str
doc_id  paragraph_num sentence_num                       
983556  0             0            0         By        by
                                   1      YANAN     yanan
                                   2       WANG      wang
                                   3        and       and
                                   4       MIKE      mike
...                                         ...       ...
1040732 0             33           7         in        in
                                   8        the       the
                                   9   adjacent  adjacent
                                   10      post      post
                                   11      here      here

[2617125 rows x 2 columns]

In [ ]:
·

In [278]:
text = df_covid.doc_content[2509]

In [279]:
para_lst = text.split('\n      ')

In [280]:
para_lst = pd.DataFrame(para_lst)
para_lst.columns = ['paragraph_str']
er = df_covid.doc_id[2509]
para_lst = pd.concat([para_lst], keys=[er], names=['doc_id'])


In [281]:
K = para_lst.paragraph_str.apply(lambda x: pd.Series(x.split('.'))).stack()

In [282]:
K = K.to_frame().rename(columns={0:'sentence_str'})

In [283]:
K.head()

sentence_str
doc_id                                                        
1035069 0 0  closeVideoCoronavirus drive-thru testing site ...
          1  New Yorkers woke up Friday in the city that ne...
          2  Subways and commuter trains were running but w...
          3  Aqueduct racetrack near JFK Airport told railb...
          4  Thursday's dizzying series of coronavirus-rela...

In [284]:
K.index.names = ['doc_id','paragraph_num','sentence_num']

In [285]:
K.head(10)

sentence_str
doc_id  paragraph_num sentence_num                                                   
1035069 0             0             closeVideoCoronavirus drive-thru testing site ...
                      1             New Yorkers woke up Friday in the city that ne...
                      2             Subways and commuter trains were running but w...
                      3             Aqueduct racetrack near JFK Airport told railb...
                      4             Thursday's dizzying series of coronavirus-rela...
                      5                                                              
        1             0             A pedestrian wearing a face mask stops in Time...
                      1              New York City Mayor Bill de Blasio said Thurs...
                      2              For most people, the new coronavirus causes o...
                      3                     For some it can cause more severe illness

In [286]:
K = K.sentence_str.apply(lambda x: pd.Series(x.split())).stack()

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [287]:
K = K.to_frame().rename(columns={0:'token_str'})

In [289]:
K.sample(20)

token_str
doc_id  paragraph_num sentence_num                
1035069 5             1            3            of
        4             4            16       behind
        3             0            21          New
        1             1            24           in
        6             7            19     positive
        3             1            0           (AP
        2             5            17  Manhattan’s
        0             4            12     cultural
        7             0            3            an
        2             5            25         home
        4             2            13    officials
        8             0            16        north
                      1            16           he
        6             6            17         they
        5             0            19    following
        6             0            9            47
                      9            5            is
                      2            6            de
        4             0            15        Park,
        2             5            16         fill

In [150]:
K['term_str'] = K.token_str.replace('\W+', '', regex=True).str.lower()

In [277]:
K.sample(20)

AttributeError: 'list' object has no attribute 'sample'